## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = attrition_df.drop(columns=['Attrition', 'Department']).columns
                    
# Create X_df using your selected columns
X_df = attrition_df[selected_columns].drop(columns=[
    'Education', 'EducationField', 'MaritalStatus', 'NumCompaniesWorked',
    'RelationshipSatisfaction']) # removing fields the company doesn't have control over

# Show the data types for X_df
X_df.dtypes


Age                         int64
BusinessTravel             object
DistanceFromHome            int64
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobRole                    object
JobSatisfaction             int64
OverTime                   object
PercentSalaryHike           int64
PerformanceRating           int64
StockOptionLevel            int64
TotalWorkingYears           int64
TrainingTimesLastYear       int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsInCurrentRole          int64
YearsSinceLastPromotion     int64
YearsWithCurrManager        int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)


In [6]:
X_train['BusinessTravel'].value_counts()

Travel_Rarely        785
Travel_Frequently    208
Non-Travel           109
Name: BusinessTravel, dtype: int64

In [7]:
# one-hot encode the BusinessTravel column
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

bt_ohe = ohe.fit_transform(X_train[['BusinessTravel']])
test_bt_ohe = ohe.transform(X_test[['BusinessTravel']])

X_train[ohe.get_feature_names_out()] = bt_ohe
X_test[ohe.get_feature_names_out()] = test_bt_ohe

X_train = X_train.drop(columns=['BusinessTravel'])
X_test = X_test.drop(columns=['BusinessTravel'])

X_train.head(2)

,Age,DistanceFromHome,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,OverTime,PercentSalaryHike,...,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely
766,50,2,2,62,3,5,Research Director,3,Yes,11,...,29,2,2,8,1,7,7,0.0,0.0,1.0
1101,32,5,4,67,2,2,Research Scientist,2,No,12,...,12,2,3,7,1,2,5,0.0,0.0,1.0


In [8]:
X_train['JobRole'].value_counts()

Sales Executive              238
Research Scientist           217
Laboratory Technician        202
Manufacturing Director       110
Healthcare Representative     98
Manager                       74
Research Director             63
Sales Representative          60
Human Resources               40
Name: JobRole, dtype: int64

In [9]:
# label encode the JobRole column
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder().fit(X_train['JobRole'])
X_train['JobRole'] = label_encoder.transform(X_train['JobRole'])
X_test['JobRole'] = label_encoder.transform(X_test['JobRole'])

In [10]:
X_train['JobRole'].value_counts()

7    238
6    217
2    202
4    110
0     98
3     74
5     63
8     60
1     40
Name: JobRole, dtype: int64

In [11]:
X_train['OverTime'].value_counts()

No     786
Yes    316
Name: OverTime, dtype: int64

In [12]:
X_train['OverTime'] = X_train['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

In [13]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# See above

In [14]:
# Create a StandardScaler
# Fit the StandardScaler to the training data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)

# Scale the training and testing data
X_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)


In [15]:
# Create a OneHotEncoder for the Department column
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
dept_ohe = ohe.fit_transform(y_train[['Department']])
test_dept_ohe = ohe.transform(y_test[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_dep = pd.DataFrame(dept_ohe, columns=ohe.get_feature_names_out())
y_test_dep = pd.DataFrame(test_dept_ohe, columns=ohe.get_feature_names_out())

y_train = y_train.drop(columns=['Department'])
y_test = y_test.drop(columns=['Department'])


In [16]:
# Create a OneHotEncoder for the Attrition column
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # drop first to reduce multicollinearity

attrition_ohe = ohe.fit_transform(y_train[['Attrition']])
test_attrition_ohe = ohe.transform(y_test[['Attrition']])

# Fit the encoder to the training data


# Create two new variables by applying the encoder
# to the training and testing data

y_train_att = pd.DataFrame(attrition_ohe, columns=ohe.get_feature_names_out())
y_test_att = pd.DataFrame(test_attrition_ohe, columns=ohe.get_feature_names_out())

display(y_train_att)


,Attrition_No,Attrition_Yes
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
1097,1.0,0.0
1098,1.0,0.0
1099,1.0,0.0
1100,1.0,0.0


## Create, Compile, and Train the Model

In [17]:
# Find the number of columns in the X training data
inputs = X_train.shape[1:]

# Create the input layer
input_layer = layers.Input(shape=(inputs))

# Create at least two shared layers
shared_layer1 = layers.Dense(128, activation='relu')(input_layer)
shared_layer2 = layers.Dense(64, activation='relu')(shared_layer1)
shared_layer3 = layers.Dense(32, activation='relu')(shared_layer2)


In [18]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(16, activation='relu')(shared_layer3)

# Create the output layer
dept_output = layers.Dense(y_train_dep.shape[1], activation='softmax', name='dept_output')(dept_hidden)


In [19]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attr_hidden = layers.Dense(16, activation='relu')(shared_layer3)

# Create the output layer
attr_output = layers.Dense(y_train_att.shape[1], activation='sigmoid', name='attr_output')(attr_hidden)


In [20]:
# Create the model
nn_model = Model(inputs=input_layer, outputs=[dept_output, attr_output])

# Compile the model
nn_model.compile(optimizer='adam',
                loss={'dept_output': 'categorical_crossentropy', 'attr_output': 'binary_crossentropy'},
                metrics={'dept_output': 'accuracy', 'attr_output': 'accuracy'})


# Summarize the model
nn_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 22)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 128)                  2944      ['input_1[0][0]']             
                                                                                                  


 dense_1 (Dense)             (None, 64)                   8256      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 32)                   2080      ['dense_1[0][0]']             
                                                                                                  
 dense_3 (Dense)             (None, 16)                   528       ['dense_2[0][0]']             
                                                                                                  
 dense_4 (Dense)             (None, 16)                   528       ['dense_2[0][0]']             
                                                                                                  
 dept_output (Dense)         (None, 3)                    51        ['dense_3[0][0]']             
                                                                                                  
 attr_outp

In [21]:
# Train the model
history = nn_model.fit(X_train, {'dept_output': y_train_dep, 'attr_output': y_train_att},
                        epochs=100, batch_size=100, verbose=1, validation_split=0.2)


Epoch 1/100
9/9 [==============================] - 3s 64ms/step - loss: 1.6749 - dept_output_loss: 0.9298 - attr_output_loss: 0.7452 - dept_output_accuracy: 0.6220 - attr_output_accuracy: 0.2633 - val_loss: 1.4844 - val_dept_output_loss: 0.7966 - val_attr_output_loss: 0.6877 - val_dept_output_accuracy: 0.6878 - val_attr_output_accuracy: 0.5928
Epoch 2/100
9/9 [==============================] - 0s 8ms/step - loss: 1.4512 - dept_output_loss: 0.8001 - attr_output_loss: 0.6511 - dept_output_accuracy: 0.6538 - attr_output_accuracy: 0.8093 - val_loss: 1.3500 - val_dept_output_loss: 0.7320 - val_attr_output_loss: 0.6180 - val_dept_output_accuracy: 0.6878 - val_attr_output_accuracy: 0.8190
Epoch 3/100
9/9 [==============================] - 0s 8ms/step - loss: 1.3384 - dept_output_loss: 0.7630 - attr_output_loss: 0.5754 - dept_output_accuracy: 0.6538 - attr_output_accuracy: 0.8434 - val_loss: 1.2661 - val_dept_output_loss: 0.7181 - val_attr_output_loss: 0.5480 - val_dept_output_accuracy: 0.6878

In [22]:
# Evaluate the model with the testing data
results = nn_model.evaluate(X_scaled_test, {'dept_output': y_test_dep, 'attr_output': y_test_att}, verbose=1)

# Print the model loss and accuracy results
print(f"Loss: {results[0]}, Department Accuracy: {results[3]}, Attrition Accuracy: {results[4]}")


12/12 [==============================] - 0s 2ms/step - loss: 2.1314 - dept_output_loss: 0.9802 - attr_output_loss: 1.1512 - dept_output_accuracy: 0.8478 - attr_output_accuracy: 0.8098
Loss: 2.13142728805542, Department Accuracy: 0.8478260636329651, Attrition Accuracy: 0.8097826242446899


In [23]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {results[3]}")
print(f"Attrition Accuracy: {results[4]}")

Department Accuracy: 0.8478260636329651
Attrition Accuracy: 0.8097826242446899


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 